# Data from stt.pecha.tools

Connect to the postgres database hosted on Render using dbeaver-ce and run the following query and export the result as a csv file. 

![export_csv](data/export_csv.png)


### All transcribed data
Use the following query when you want more data but willing to compromise one the quality of the transcription. Task in state 'submitted' means the task has been transcribed by the first transcriber and not yet been reviewed or done quality control on.
```sql
select t.transcript, t.reviewed_transcript, t.final_transcript, t.file_name, t.url 
from "Task" t 
where t.state in ('submitted', 'accepted', 'finalised');
```

### Reviewed only
The following query selects only the task that has been reviewed by the team lead but may or may not have been reviewed by the quality control team.
reviewed at least once
```sql 
select t.reviewed_transcript, t.final_transcript, t.file_name, t.url 
from "Task" t 
where t.state in ('accepted', 'finalised');
```

Read the exported csv from stt.pecha.tools

In [1]:
import pandas as pd

csv_path = "/home/monlamai/_Task__202405241257.csv"

df = pd.read_csv(csv_path)

In [2]:
df.head()

,reviewed_transcript,final_transcript,file_name,url
0,དེ་ལ་བརྟེན་ནས་མེ་དང་བུད་ཤིང་ལ་རང་བཞིན་མེད་དོ་ས...,NaN,STT_AB00565_0410_1675759_to_1682175,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...
1,ཐལ་འགྱུར་ནི་,NaN,STT_AB00565_0412_1692363_to_1693165,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...
2,ཕ་རོལ་པོ་ལ་མི་འདོད་པ་འཕེན་པ་འདི་འདྲ་རྣམས་ཡིན་པས་,NaN,STT_AB00565_0415_1709513_to_1713421,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...
3,དེས་མཚོན་ནས་ཐལ་བ་གཞན་རྣམས་ཀྱང་རྟོགས་པར་བྱའོ། །,NaN,STT_AB00565_0416_1714087_to_1717790,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...
4,དཔང་པོ་རྒོལ་བ་དང་,NaN,STT_AB00565_0424_1767704_to_1769104,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...


In [3]:
! mkdir tsv
! curl https://d38pmlk0v88drf.cloudfront.net/backup/finalised_2024-03-20.csv --output tsv/finalised_2024-03-20.csv
! curl https://d38pmlk0v88drf.cloudfront.net/backup/finalised_2024-04-24.csv --output tsv/finalised_2024-04-24.csv

mkdir: cannot create directory ‘tsv’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  205M  100  205M    0     0  10.9M      0  0:00:18  0:00:18 --:--:-- 10.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  173M  100  173M    0     0  11.1M      0  0:00:15  0:00:15 --:--:-- 11.5M    0     0  11.0M      0  0:00:15  0:00:13  0:00:02 11.0M


In [4]:
df_backups = []
for backup in ['2024-03-20', '2024-04-24']:
    df_backups.append(pd.read_csv(f'tsv/finalised_{backup}.csv'))

In [11]:
df_backup = pd.concat(df_backups)

In [12]:
df_backup = df_backup[['reviewed_transcript','final_transcript','file_name','url']]

In [14]:
df = pd.concat([df, df_backup])

In [15]:
df.head()

,reviewed_transcript,final_transcript,file_name,url
0,དེ་ལ་བརྟེན་ནས་མེ་དང་བུད་ཤིང་ལ་རང་བཞིན་མེད་དོ་ས...,NaN,STT_AB00565_0410_1675759_to_1682175,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...
1,ཐལ་འགྱུར་ནི་,NaN,STT_AB00565_0412_1692363_to_1693165,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...
2,ཕ་རོལ་པོ་ལ་མི་འདོད་པ་འཕེན་པ་འདི་འདྲ་རྣམས་ཡིན་པས་,NaN,STT_AB00565_0415_1709513_to_1713421,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...
3,དེས་མཚོན་ནས་ཐལ་བ་གཞན་རྣམས་ཀྱང་རྟོགས་པར་བྱའོ། །,NaN,STT_AB00565_0416_1714087_to_1717790,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...
4,དཔང་པོ་རྒོལ་བ་དང་,NaN,STT_AB00565_0424_1767704_to_1769104,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...


grade column is used to indicate if the quality of the transcription 
|grade|meaning|
|-----|--------|
| 1   | Transcribed only|
| 2   | Reviewed by the team lead|
| 3   | Reviewed by the quality control team|

In [16]:
df['grade'] = 1 # by default all the task are transcibed

In [17]:
df.loc[~df['reviewed_transcript'].isna(), 'grade'] = 2 # if the task has been reviewed, grade = 2

In [18]:
df.loc[~df['final_transcript'].isna(), 'grade'] = 3 # if the task has been finalized, grade = 3

In [19]:
df['final_transcript'].fillna(df['reviewed_transcript'], inplace=True)

In [20]:
df.head()

,reviewed_transcript,final_transcript,file_name,url,grade
0,དེ་ལ་བརྟེན་ནས་མེ་དང་བུད་ཤིང་ལ་རང་བཞིན་མེད་དོ་ས...,དེ་ལ་བརྟེན་ནས་མེ་དང་བུད་ཤིང་ལ་རང་བཞིན་མེད་དོ་ས...,STT_AB00565_0410_1675759_to_1682175,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2
1,ཐལ་འགྱུར་ནི་,ཐལ་འགྱུར་ནི་,STT_AB00565_0412_1692363_to_1693165,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2
2,ཕ་རོལ་པོ་ལ་མི་འདོད་པ་འཕེན་པ་འདི་འདྲ་རྣམས་ཡིན་པས་,ཕ་རོལ་པོ་ལ་མི་འདོད་པ་འཕེན་པ་འདི་འདྲ་རྣམས་ཡིན་པས་,STT_AB00565_0415_1709513_to_1713421,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2
3,དེས་མཚོན་ནས་ཐལ་བ་གཞན་རྣམས་ཀྱང་རྟོགས་པར་བྱའོ། །,དེས་མཚོན་ནས་ཐལ་བ་གཞན་རྣམས་ཀྱང་རྟོགས་པར་བྱའོ། །,STT_AB00565_0416_1714087_to_1717790,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2
4,དཔང་པོ་རྒོལ་བ་དང་,དཔང་པོ་རྒོལ་བ་དང་,STT_AB00565_0424_1767704_to_1769104,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,2


In [21]:
df = df[~df['final_transcript'].isnull()]

In [22]:
df['dept'] = df['file_name'].str[:6] # extract department code from file name

In [23]:
df = df[['file_name','final_transcript', 'url', 'dept', 'grade']]

In [24]:
df.rename(columns={'final_transcript':'uni'}, inplace=True)

In [25]:
df.head()

,file_name,uni,url,dept,grade
0,STT_AB00565_0410_1675759_to_1682175,དེ་ལ་བརྟེན་ནས་མེ་དང་བུད་ཤིང་ལ་རང་བཞིན་མེད་དོ་ས...,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,STT_AB,2
1,STT_AB00565_0412_1692363_to_1693165,ཐལ་འགྱུར་ནི་,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,STT_AB,2
2,STT_AB00565_0415_1709513_to_1713421,ཕ་རོལ་པོ་ལ་མི་འདོད་པ་འཕེན་པ་འདི་འདྲ་རྣམས་ཡིན་པས་,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,STT_AB,2
3,STT_AB00565_0416_1714087_to_1717790,དེས་མཚོན་ནས་ཐལ་བ་གཞན་རྣམས་ཀྱང་རྟོགས་པར་བྱའོ། །,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,STT_AB,2
4,STT_AB00565_0424_1767704_to_1769104,དཔང་པོ་རྒོལ་བ་དང་,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,STT_AB,2


In [26]:
df['file_name'] = df['file_name'].map(lambda x: x.replace(".wav", ""))
df['file_name'] = df['file_name'].map(lambda x: x.replace(".mp3", ""))

In [27]:
df.to_csv('01_stt_pecha_tools.csv', index=False)

In [28]:
! aws s3 cp 01_stt_pecha_tools.csv s3://monlam.ai.stt/tsv/01_stt_pecha_tools.csv

upload: ./01_stt_pecha_tools.csv to s3://monlam.ai.stt/tsv/01_stt_pecha_tools.csv


In [29]:
df.shape

(576532, 5)